In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder , MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense , Dropout
from keras.optimizers import RMSprop , adam
from keras.utils import to_categorical

import tensorflow as tf
from tensorflow.python.platform import flags

from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score , roc_curve , auc , f1_score
from sklearn.preprocessing import LabelEncoder , MinMaxScaler
from sklearn.svm import SVC , LinearSVC

import matplotlib . pyplot as plt
plt.style.use('bmh')

FLAGS = flags.FLAGS

flags.DEFINE_integer ("nb_epochs", 20 , "Number of epochs to train model")
flags.DEFINE_integer ("batch_size", 128 , "Size of training batches")
flags.DEFINE_float ("learning_rate", 0.1 , "Learning rate for training")
flags.DEFINE_integer ("nb_classes" , 5 , "Number of classification classes")
flags.DEFINE_integer ("source_samples" , 10 , "Nb of test set examples to attack")

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

In [ ]:
spark = SparkSession.builder.appName("cybersecurity").getOrCreate()
df_test = spark.read.csv('KDDTest+.csv',inferSchema=True,header=False).toDF("duration", "protocol" , "service" , "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", 
         "hot", "num_failed_logins", "logged_in ", "num_compromised", "root_shell", "su_attempted", "num_root", 
         "num_file_creations", "num_shells", "num_access_files","num_outbound_cmds ","is_host_login ", "is_guest_login",
         "count", "srv_count", "serror_rate", "srv_serror_rate","rerror_rate", "srv_rerror_rate", "same_srv_rate", 
         "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", " dst_host_srv_count", "dst_host_same_srv_rate", 
         "dst_host_diff_srv_rate","dst_host_same_src_port_rate", " dst_host_srv_diff_host_rate", "dst_host_serror_rate",
         "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "attack_type", "other")
df_train = spark.read.csv('KDDTrain+.csv',inferSchema=True,header=False).toDF("duration", "protocol" , "service" , "flag", "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent", 
         "hot", "num_failed_logins", "logged_in ", "num_compromised", "root_shell", "su_attempted", "num_root", 
         "num_file_creations", "num_shells", "num_access_files","num_outbound_cmds ","is_host_login ", "is_guest_login",
         "count", "srv_count", "serror_rate", "srv_serror_rate","rerror_rate", "srv_rerror_rate", "same_srv_rate", 
         "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", " dst_host_srv_count", "dst_host_same_srv_rate", 
         "dst_host_diff_srv_rate","dst_host_same_src_port_rate", " dst_host_srv_diff_host_rate", "dst_host_serror_rate",
         "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "attack_type", "other")

In [ ]:

df = df_train.union(df_test)


In [ ]:
df = df.na.replace(["neptune", "back", "land", "pod", "smurf", "teardrop", "mailbomb", "processtable", "udpstorm", "apache2", "worm"], "dos", "attack_type")

df = df.na.replace(["buffer_overflow ", "loadmodule", "perl", "rootkit", "sqlattack", "xterm", "ps"], "u2r", "attack_type")

df = df.na.replace(["ftp_write", "guess_passwd", "imap", "multihop", "phf", "spy", "warezclient", "warezmaster", "xlock", "xsnoop", "snmpgetattack", "httptunnel", "snmpguess", "sendmail", "named"], "r2l", "attack_type")

df = df.na.replace(["satan ", "ipsweep", "nmap", "portsweep", "saint", "mscan"], "probe", "attack_type")


In [ ]:
df[df.attack_type == "u2r"].show()

In [ ]:
X = df.select(df.columns[:-2])
y = df.select(df.columns[41])

In [ ]:
stringIndexer = StringIndexer(inputCol="attack_type", outputCol="attackIndex")
model = stringIndexer.fit(y)
indexed = model.transform(y)

encoder = OneHotEncoder(inputCol="attackIndex", outputCol="attack_type_Vec")
encoded = encoder.transform(indexed)
encoded.show()


In [ ]:
type(encoded)

In [ ]:
y = encoded.select(encoded.columns[2])

In [ ]:
y.show()